In [1]:
# ============================================
# ÉTAPE 2 : PREPROCESSING DES DONNÉES
# Projet : Estimation de prix immobilier
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("✅ Librairies importées avec succès")

✅ Librairies importées avec succès


In [2]:
# ============================================
# 2.1 CHARGEMENT DES DONNÉES
# ============================================

path = "../Data/raw/"

# Chargement des données principales
transactions = pd.read_csv(path + "transactions_sample.csv")
loyers = pd.read_csv(path + "loyers.csv")
foyers_fiscaux = pd.read_csv(path + "foyers_fiscaux.csv")
parc_immobilier = pd.read_csv(path + "parc_immobilier.csv")

print(f"✅ Données chargées:")
print(f"   - Transactions: {transactions.shape}")
print(f"   - Loyers: {loyers.shape}")
print(f"   - Foyers fiscaux: {foyers_fiscaux.shape}")
print(f"   - Parc immobilier: {parc_immobilier.shape}")

✅ Données chargées:
   - Transactions: (100, 20)
   - Loyers: (105391, 6)
   - Foyers fiscaux: (315542, 15)
   - Parc immobilier: (48627, 6)


In [3]:
# ============================================
# 2.2 NETTOYAGE DU DATASET TRANSACTIONS
# ============================================

print("="*60)
print("🧹 NETTOYAGE DES TRANSACTIONS")
print("="*60)

# Copie de travail
df = transactions.copy()

# Conversion de la date
df['date_transaction'] = pd.to_datetime(df['date_transaction'])
df['annee'] = df['date_transaction'].dt.year

# Suppression des colonnes non pertinentes pour le modèle
cols_to_drop = ['id_transaction', 'adresse', 'id_parcelle_cadastre', 
                'surface_dependances', 'surface_locaux_industriels',
                'surface_terrains_agricoles', 'surface_terrains_sols', 
                'surface_terrains_nature']
df = df.drop(columns=cols_to_drop, errors='ignore')

# Encodage du type de bâtiment
df['type_batiment_encoded'] = df['type_batiment'].map({'Maison': 1, 'Appartement': 0})

# Encodage VEFA (Vente en État Futur d'Achèvement)
df['vefa_encoded'] = df['vefa'].astype(int)

print(f"\n✅ Colonnes après nettoyage: {list(df.columns)}")
print(f"✅ Shape: {df.shape}")
display(df.head())

🧹 NETTOYAGE DES TRANSACTIONS

✅ Colonnes après nettoyage: ['date_transaction', 'prix', 'departement', 'id_ville', 'ville', 'code_postal', 'type_batiment', 'vefa', 'n_pieces', 'surface_habitable', 'latitude', 'longitude', 'annee', 'type_batiment_encoded', 'vefa_encoded']
✅ Shape: (100, 15)


,date_transaction,prix,departement,id_ville,ville,code_postal,type_batiment,vefa,n_pieces,surface_habitable,latitude,longitude,annee,type_batiment_encoded,vefa_encoded
0,2015-07-22,222500.0,63,247,MUROL,63790,Maison,False,4,123,45.572973,2.949976,2015,1,0
1,2024-06-18,218640.0,73,15,LES ALLUES,73550,Appartement,False,1,23,45.398740,6.567602,2024,0,0
2,2020-07-23,254950.0,77,316,MORET-LOING-ET-ORVANNE,77250,Maison,False,6,124,48.333762,2.780830,2020,1,0
3,2022-10-03,380000.0,94,81,VITRY-SUR-SEINE,94400,Maison,False,3,93,48.796988,2.375558,2022,1,0
4,2018-07-12,258000.0,77,284,MEAUX,77100,Maison,False,5,96,48.948816,2.891734,2018,1,0


In [6]:
# ============================================
# 2.3 ENRICHISSEMENT AVEC LES LOYERS LOCAUX
# ============================================

print("="*60)
print("🔗 ENRICHISSEMENT AVEC LES LOYERS")
print("="*60)

# Convertir les départements en string pour la fusion (gestion 2A, 2B)
loyers['departement'] = loyers['departement'].astype(str)
df['departement'] = df['departement'].astype(str)

# Préparer les loyers (moyenne par ville)
loyers_recent = loyers.groupby(['departement', 'id_ville']).agg({
    'loyer_m2_appartement': 'mean',
    'loyer_m2_maison': 'mean'
}).reset_index()

# Renommer les colonnes
loyers_recent.columns = ['departement', 'id_ville', 'loyer_m2_appart_local', 'loyer_m2_maison_local']

# Fusion avec les transactions
df = df.merge(loyers_recent, on=['departement', 'id_ville'], how='left')

# Créer une variable de loyer selon le type de bien
df['loyer_m2_local'] = np.where(
    df['type_batiment'] == 'Appartement',
    df['loyer_m2_appart_local'],
    df['loyer_m2_maison_local']
)

print(f"✅ Loyers ajoutés!")
print(f"   Valeurs manquantes loyer_m2_local: {df['loyer_m2_local'].isna().sum()}")
print(f"\n📊 Statistiques des loyers locaux:")
print(df[['loyer_m2_appart_local', 'loyer_m2_maison_local', 'loyer_m2_local']].describe())

🔗 ENRICHISSEMENT AVEC LES LOYERS
✅ Loyers ajoutés!
   Valeurs manquantes loyer_m2_local: 2

📊 Statistiques des loyers locaux:
       loyer_m2_appart_local  loyer_m2_maison_local  loyer_m2_local
count              98.000000              98.000000       98.000000
mean               12.384274              11.165277       11.634898
std                 3.882798               4.068419        4.286624
min                 7.181524               6.261501        6.261501
25%                 9.888437               8.407548        8.411336
50%                11.573230              10.246154       10.738296
75%                14.064499              12.693911       13.806236
max                27.106262              25.028557       27.106262


In [7]:
# ============================================
# 2.4 ENRICHISSEMENT AVEC LES DONNÉES FISCALES
# ============================================

print("="*60)
print("🔗 ENRICHISSEMENT AVEC LES DONNÉES FISCALES")
print("="*60)

# Convertir le département en string
foyers_fiscaux['departement'] = foyers_fiscaux['departement'].astype(str)

# Préparer les données fiscales (dernière année par ville)
fiscal_recent = foyers_fiscaux.groupby(['departement', 'id_ville']).agg({
    'revenu_fiscal_moyen': 'mean',
    'n_foyers_fiscaux': 'mean'
}).reset_index()

# Fusion avec les transactions
df = df.merge(fiscal_recent, on=['departement', 'id_ville'], how='left')

print(f"✅ Données fiscales ajoutées!")
print(f"   Valeurs manquantes revenu_fiscal_moyen: {df['revenu_fiscal_moyen'].isna().sum()}")
print(f"\n📊 Statistiques du revenu fiscal moyen:")
print(df['revenu_fiscal_moyen'].describe())

🔗 ENRICHISSEMENT AVEC LES DONNÉES FISCALES
✅ Données fiscales ajoutées!
   Valeurs manquantes revenu_fiscal_moyen: 4

📊 Statistiques du revenu fiscal moyen:
count       96.000000
mean     28489.032371
std       8350.438602
min      15395.831111
25%      22897.753611
50%      26046.380556
75%      32108.307778
max      55557.645000
Name: revenu_fiscal_moyen, dtype: float64


In [8]:
# ============================================
# 2.5 GESTION DES VALEURS MANQUANTES
# ============================================

print("="*60)
print("❓ GESTION DES VALEURS MANQUANTES")
print("="*60)

# Afficher les valeurs manquantes
missing = df.isnull().sum()
print("\n📊 Valeurs manquantes par colonne:")
print(missing[missing > 0])

# Imputation par la médiane pour les variables numériques
df['loyer_m2_local'] = df['loyer_m2_local'].fillna(df['loyer_m2_local'].median())
df['loyer_m2_appart_local'] = df['loyer_m2_appart_local'].fillna(df['loyer_m2_appart_local'].median())
df['loyer_m2_maison_local'] = df['loyer_m2_maison_local'].fillna(df['loyer_m2_maison_local'].median())
df['revenu_fiscal_moyen'] = df['revenu_fiscal_moyen'].fillna(df['revenu_fiscal_moyen'].median())
df['n_foyers_fiscaux'] = df['n_foyers_fiscaux'].fillna(df['n_foyers_fiscaux'].median())

# Vérification
print("\n✅ Après imputation:")
print(f"   Valeurs manquantes totales: {df.isnull().sum().sum()}")

❓ GESTION DES VALEURS MANQUANTES

📊 Valeurs manquantes par colonne:
loyer_m2_appart_local    2
loyer_m2_maison_local    2
loyer_m2_local           2
revenu_fiscal_moyen      4
n_foyers_fiscaux         4
dtype: int64

✅ Après imputation:
   Valeurs manquantes totales: 0


In [9]:
# ============================================
# 2.6 FEATURE ENGINEERING
# ============================================

print("="*60)
print("🔧 FEATURE ENGINEERING")
print("="*60)

# Prix au m²
df['prix_m2'] = df['prix'] / df['surface_habitable']

# Ratio loyer/prix (rendement locatif potentiel)
df['rendement_locatif'] = (df['loyer_m2_local'] * 12) / df['prix_m2'] * 100

# Surface par pièce
df['surface_par_piece'] = df['surface_habitable'] / df['n_pieces'].replace(0, 1)

# Indicateur région parisienne (départements 75, 77, 78, 91, 92, 93, 94, 95)
idf_depts = ['75', '77', '78', '91', '92', '93', '94', '95']
df['is_idf'] = df['departement'].isin(idf_depts).astype(int)

print("✅ Nouvelles features créées:")
print(f"   - prix_m2: Prix au m²")
print(f"   - rendement_locatif: Rendement locatif potentiel (%)")
print(f"   - surface_par_piece: Surface moyenne par pièce")
print(f"   - is_idf: Indicateur Île-de-France")

print(f"\n📊 Aperçu des nouvelles features:")
display(df[['prix', 'surface_habitable', 'prix_m2', 'rendement_locatif', 'surface_par_piece', 'is_idf']].describe())

🔧 FEATURE ENGINEERING
✅ Nouvelles features créées:
   - prix_m2: Prix au m²
   - rendement_locatif: Rendement locatif potentiel (%)
   - surface_par_piece: Surface moyenne par pièce
   - is_idf: Indicateur Île-de-France

📊 Aperçu des nouvelles features:


,prix,surface_habitable,prix_m2,rendement_locatif,surface_par_piece,is_idf
count,1.000000e+02,100.000000,100.000000,100.000000,100.000000,100.000000
mean,2.088397e+05,78.380000,2822.725891,18.107943,22.691738,0.170000
std,1.759325e+05,37.577067,1968.390322,106.344346,5.803357,0.377525
min,5.000000e+02,23.000000,14.705882,1.317174,10.666667,0.000000
25%,1.192500e+05,48.750000,1708.710407,4.175441,18.500000,0.000000
50%,1.690000e+05,77.500000,2299.074074,5.388657,21.166667,0.000000
75%,2.420608e+05,96.250000,3577.332242,7.286267,25.687500,0.000000
max,1.271260e+06,204.000000,9867.567568,1065.391124,43.500000,1.000000


In [10]:
# ============================================
# 2.7 PRÉPARATION DU DATASET FINAL POUR ML
# ============================================

print("="*60)
print("📦 PRÉPARATION DU DATASET FINAL")
print("="*60)

# Sélection des features pour le modèle
features = [
    'surface_habitable',
    'n_pieces',
    'type_batiment_encoded',
    'vefa_encoded',
    'loyer_m2_local',
    'revenu_fiscal_moyen',
    'surface_par_piece',
    'is_idf',
    'latitude',
    'longitude'
]

target = 'prix'

# Création du dataset final
X = df[features]
y = df[target]

print(f"✅ Dataset prêt pour le ML:")
print(f"   - Features (X): {X.shape}")
print(f"   - Target (y): {y.shape}")
print(f"\n📋 Liste des features:")
for i, f in enumerate(features, 1):
    print(f"   {i}. {f}")

# Sauvegarde du dataset préprocessé
df_final = df[features + [target] + ['ville', 'departement', 'type_batiment']]
df_final.to_csv("../Data/raw/transactions_preprocessed.csv", index=False)
print(f"\n✅ Dataset sauvegardé: transactions_preprocessed.csv")

📦 PRÉPARATION DU DATASET FINAL
✅ Dataset prêt pour le ML:
   - Features (X): (100, 10)
   - Target (y): (100,)

📋 Liste des features:
   1. surface_habitable
   2. n_pieces
   3. type_batiment_encoded
   4. vefa_encoded
   5. loyer_m2_local
   6. revenu_fiscal_moyen
   7. surface_par_piece
   8. is_idf
   9. latitude
   10. longitude

✅ Dataset sauvegardé: transactions_preprocessed.csv


In [11]:
# ============================================
# 2.8 RÉSUMÉ DU PREPROCESSING
# ============================================

print("="*70)
print("📋 RÉSUMÉ DU PREPROCESSING")
print("="*70)

print("""
✅ ÉTAPES RÉALISÉES:
   1. Nettoyage des colonnes non pertinentes
   2. Encodage des variables catégorielles (type_batiment, vefa)
   3. Enrichissement avec les loyers locaux par ville
   4. Enrichissement avec les données fiscales (revenu moyen)
   5. Imputation des valeurs manquantes (médiane)
   6. Feature engineering:
      - Prix au m²
      - Rendement locatif potentiel
      - Surface par pièce
      - Indicateur Île-de-France

📊 DATASET FINAL:
   - 100 observations
   - 10 features
   - 1 variable cible (prix)

🔧 FEATURES SÉLECTIONNÉES:
   - surface_habitable: Surface en m²
   - n_pieces: Nombre de pièces
   - type_batiment_encoded: Maison(1) / Appartement(0)
   - vefa_encoded: VEFA oui(1) / non(0)
   - loyer_m2_local: Loyer moyen local au m²
   - revenu_fiscal_moyen: Revenu fiscal moyen de la ville
   - surface_par_piece: Surface moyenne par pièce
   - is_idf: Localisation en Île-de-France
   - latitude, longitude: Coordonnées géographiques
""")

print("="*70)
print("✅ PROCHAINE ÉTAPE: MODÉLISATION ML")
print("="*70)

📋 RÉSUMÉ DU PREPROCESSING

✅ ÉTAPES RÉALISÉES:
   1. Nettoyage des colonnes non pertinentes
   2. Encodage des variables catégorielles (type_batiment, vefa)
   3. Enrichissement avec les loyers locaux par ville
   4. Enrichissement avec les données fiscales (revenu moyen)
   5. Imputation des valeurs manquantes (médiane)
   6. Feature engineering:
      - Prix au m²
      - Rendement locatif potentiel
      - Surface par pièce
      - Indicateur Île-de-France

📊 DATASET FINAL:
   - 100 observations
   - 10 features
   - 1 variable cible (prix)

🔧 FEATURES SÉLECTIONNÉES:
   - surface_habitable: Surface en m²
   - n_pieces: Nombre de pièces
   - type_batiment_encoded: Maison(1) / Appartement(0)
   - vefa_encoded: VEFA oui(1) / non(0)
   - loyer_m2_local: Loyer moyen local au m²
   - revenu_fiscal_moyen: Revenu fiscal moyen de la ville
   - surface_par_piece: Surface moyenne par pièce
   - is_idf: Localisation en Île-de-France
   - latitude, longitude: Coordonnées géographiques

✅ PROCHAI